#Dense Neural Network

Begin by importing the necessary libraries. Seeds are set for reproducibility.

In [1]:
import numpy as np
np.random.seed(1234)
from keras.models import Sequential
import tensorflow as tf
tf.random.set_seed(123)
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

Load and preprocess the MNIST data set.

In [2]:
from keras.datasets import mnist

# Load and split data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#Transform 3D X data into 2D 
X_train_flatten = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])
X_test_flatten = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])

#Normalise X data elements to all be in range [0,1]
X_train_flatten = X_train_flatten.astype('float32')
X_test_flatten = X_test_flatten.astype('float32')
X_train_flatten /= 255
X_test_flatten /= 255

#Adjust y data so that numerical categorical labels become one-hot vectors of size 10
Y_train_class = np_utils.to_categorical(y_train, 10)
Y_test_class = np_utils.to_categorical(y_test, 10)

11501568/11490434 [==============================] - 0s 0us/step


Create a function to generate a 3-layer dense neural network, with parameterisable structure, to perform the prediction task.

In [3]:
INPUT_DIM = 784

def create_network(L1_neurons, L2_neurons, **kwargs):
  L1_activation = 'relu'
  L2_activation = 'relu'
  L3_activation = 'softmax'

  L3_neurons = 10

  if 'L1_activation' in kwargs.keys():
    L1_activation = kwargs['L1_activation']
  if 'L2_activation' in kwargs.keys():
    L2_activation = kwargs['L2_activation']
  if 'L3_activation' in kwargs.keys():
    L3_activation = kwargs['L3_activation']
  if 'L3_neurons' in kwargs.keys():
    L3_neurons = kwargs['L3_neurons']

  model = Sequential()
  model.add(Dense(L1_neurons, activation=L1_activation, input_shape=(INPUT_DIM,), name='L1'))
  model.add(Dense(L2_neurons, activation=L2_activation, name='L2'))
  model.add(Dense(L3_neurons, activation=L3_activation, name='L3'))

  return model

test_model = create_network(64, 32, L3_neurons = 25)
test_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (None, 64)                50240     
                                                                 
 L2 (Dense)                  (None, 32)                2080      
                                                                 
 L3 (Dense)                  (None, 25)                825       
                                                                 
Total params: 53,145
Trainable params: 53,145
Non-trainable params: 0
_________________________________________________________________


Write a function which will accept the hyperparameters, generate the neural network and evaluate it against the data set.

In [6]:
def evaluate(L1_neurons, L2_neurons, **kwargs):
  model = create_network(L1_neurons, L2_neurons, **kwargs)

  loss_function = 'categorical_crossentropy'
  optimizer = 'adam'
  if 'loss_function' in kwargs.keys():
    loss_function = kwargs['loss_function']
  if 'optimizer' in kwargs.keys():
    optimizer = kwargs['optimizer']

  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

  batch_size = 32
  if 'batch_size' in kwargs.keys():
    batch_size = kwargs['batch_size']

  history = None
  if 'epochs' in kwargs.keys():
    epochs = kwargs['epochs']
    history = model.fit(X_train_flatten, Y_train_class, batch_size=batch_size, epochs=epochs, verbose=0)
  else:
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    history = model.fit(X_train_flatten, Y_train_class, epochs=1000, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

  score = model.evaluate(X_test_flatten, Y_test_class, verbose=0)

  return {
      'loss': score[0],
      'accuracy': score[1]
  }

evaluate(64, 32, epochs=10)

{'accuracy': 0.9728000164031982, 'loss': 0.10417211055755615}

Run the tensor completion tests for speed and max memory.

In [ ]:
#IMPORTS==============================================
import unittest
import json
import numpy as np

from generateerrortensor import generateIncompleteErrorTensor
from trainmodels import evaluationFunctionGenerator, crossValidationFunctionGenerator
from loaddata import loadData, trainTestSplit, extractZeroOneClasses, convertZeroOne
from tensorcompletion import tensorcomplete_TMac_TT
from tensorcompletion import ket_augmentation, inverse_ket_augmentation
from tensorsearch import findBestValues, hyperparametersFromIndices
import regressionmetrics
import classificationmetrics

quantity = 'CPU-TIME'

known_fraction = 0.25

#Load dataset
task = 'classification'
data = loadData(source='sklearn', identifier='breast_cancer', task=task)
data_split = trainTestSplit(data, method = 'cross_validation')

budget_type = 'features'
budget_fraction = 0.5
func = crossValidationFunctionGenerator(data_split, algorithm='svm-rbf', task=task, budget_type=budget_type, budget_fraction=budget_fraction)

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

#Generate range dictionary
ranges_dict = {
    'L1_neurons': {
        'start': 0.05,
        'end': 5.00,
        'interval': 0.05,
        },
    'L2_neurons': {
        'start': 0.05,
        'end': 5.00,
        'interval': 0.05,
        },
    'L1_activation': ['relu'],
    'L2_activation': ['relu'],
    'optimizer': ['adam'],
    'batch_size':
    }

#Generate incomplete tensor
incomplete_tensor, known_indices = generateIncompleteErrorTensor(func, ranges_dict, known_fraction, metric=classificationmetrics.indicatorFunction, eval_trials=1)
print('TENSOR GENERATED')

expected_rank = [1]
#Apply tensor completion
predicted_tensor, _, _ = tensorcomplete_TMac_TT(incomplete_tensor, known_indices, expected_rank, convergence_tolerance=1e-15, iteration_limit=100000)

#Find best hyperparameter value
result = findBestValues(predicted_tensor, smallest=True, number_of_values=1)
values, indices = result['values'], result['indices']
hyperparameter_values = hyperparametersFromIndices(indices, ranges_dict)

#End timer/memory profiler/CPU timer
result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()

#Recreate cross-validation generator
data_split = trainTestSplit(data, method = 'cross_validation')
#Find the true loss for the selcted combination
truefunc = crossValidationFunctionGenerator(data_split, algorithm='svm-rbf', task=task)
true_value = truefunc(metric=classificationmetrics.indicatorFunction, **hyperparameter_values[0])

print(f'hyperparameters: {hyperparameter_values}')
print(f'function values: {values}')
print(f'True value: {true_value}')
print(f'{quantity}: {result}')